In [2]:
!pip install mlxtend

In [3]:
from tkinter import *
from tkinter import ttk, filedialog
from tkinter.filedialog import askopenfile
from tkinter import messagebox 
import numpy as np 
import pandas as pd 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Create an instance of tkinter frame
win = Tk()

# Set the geometry of tkinter frame
win.geometry("700x350")

def open_file():
   file = filedialog.askopenfile(mode='r', filetypes=[('All Files', '*.csv')])
   if file:   
      data = pd.read_csv(file)
      data['Description'] = data['Description'].str.strip()
      data.dropna(axis=0, subset=['Invoice'], inplace=True)
      data['Invoice'] = data['Invoice'].astype('str')
      data = data[~data['Invoice'].str.contains('C')]
      data_plus=data[data["Quantity"]>=0]
      basket = (data[data['Country'] =="United_Kingdom"]
          .groupby(['Invoice', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Invoice'))
      def encode_units(x):
          if x <= 0:
              return 0
          if x >= 1:
              return 1
      basket_sets = basket.applymap(encode_units)
      basket_filter=basket_sets[(basket_sets > 0).sum(axis=1) >=2]
      frq_sets=apriori(basket_filter, min_support=0.03, 
                 use_colnames=True).sort_values('support',ascending=False).reset_index(drop=True)
      frq_sets['length']=frq_sets['itemsets'].apply(lambda x: len(x))
      rules=association_rules(frq_sets, metric='lift', min_threshold=1).sort_values('lift',ascending=False).reset_index(drop=True)
      result=rules.head()
      print(result)
      file.close()
       

# Add a Label widget
label = Label(win, text="Click the Button to browse the Files", font=('Georgia 13'))
label.pack(pady=10)
# Create a Button
ttk.Button(win, text="Browse", command=open_file).pack(pady=20)


win.mainloop()

<ipython-input-3-38527d8ee892>:52: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  win.mainloop()


                           antecedents                          consequents  \
0  (WOODEN PICTURE FRAME WHITE FINISH)         (WOODEN FRAME ANTIQUE WHITE)   
1         (WOODEN FRAME ANTIQUE WHITE)  (WOODEN PICTURE FRAME WHITE FINISH)   
2     (STRAWBERRY CERAMIC TRINKET BOX)     (SWEETHEART CERAMIC TRINKET BOX)   
3     (SWEETHEART CERAMIC TRINKET BOX)     (STRAWBERRY CERAMIC TRINKET BOX)   
4              (HEART OF WICKER SMALL)              (HEART OF WICKER LARGE)   

   antecedent support  consequent support   support  confidence       lift  \
0            0.052644            0.063633  0.034060    0.646995  10.167642   
1            0.063633            0.052644  0.034060    0.535262  10.167642   
2            0.088545            0.054945  0.043553    0.491878   8.952177   
3            0.054945            0.088545  0.043553    0.792670   8.952177   
4            0.057764            0.062194  0.031414    0.543825   8.743957   

   leverage  conviction  
0  0.030710    2.652558  
1  0